In [36]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
from typing import Tuple, List 
import numpy as np
from sklearn.metrics import f1_score
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.dataloader import default_collate

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Task 1: Simple Bidirectional LSTM model (40
points)
The first task is to build a simple bidirectional LSTM model (see slides page
43 in lecture 12 for the network architecture) for NER.
Task. Implementing the bidirectional LSTM network with PyTorch. The
architecture of the network is:
Embedding →BLSTM →Linear →ELU →classifier
The hyper-parameters of the network are listed in the following table:

embedding dim 100
number of LSTM layers 1
LSTM hidden dim 256
LSTM Dropout 0.33
Linear output dim 128

Train this simple BLSTM model with the training data on NER with SGD
as the optimizer. Please tune other parameters that are not specified in the
above table, such as batch size, learning rate and learning rate scheduling.
What are the precision, recall and F1 score on the dev data? (hint: the
reasonable F1 score on dev is 77%.

In [27]:
# class BILSTM(nn.Module):
#     def __init__(self, input_dim, hidden_dim, is_bid, output_dim=1, num_layers=1, dropout=0.5):
#         super(BinaryClassificationModelWithAttention, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.is_bid = is_bid #### NEXT STEPS USE THIS
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True, bidirectional = is_bid, dropout = dropout if num_layers > 1 else 0)
#         self.dropout = nn.Dropout(dropout)
        
#         if self.is_bid:
#             self.attention = Attention(hidden_dim * 2)  
#             self.fc = nn.Linear(hidden_dim * 2, output_dim)  
#             self.sigmoid = nn.Sigmoid()
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x):
#         if self.is_bid:
#             h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(x.device)  
#             c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(x.device)  

#         lstm_output, _ = self.lstm(x, (h0, c0))
#         # context_vector, attention_weights = self.attention(lstm_output)
        
#         out = self.dropout(lstm_output)  # Apply dropout to the context vector
#         out = self.fc(out)
#         out = self.sigmoid(out)
#         return out

# # Citation https://www.kaggle.com/code/khalildmk/simple-two-layer-bidirectional-lstm-with-pytorch

# class Bi_RNN(nn.Module):

#     def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers=1, rnn_type='LSTM'):
#         super(Bi_RNN, self).__init__()
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim
#         self.batch_size = batch_size
#         self.num_layers = num_layers
#         self.dropout = nn.Dropout(0.33) 
#         self.init_linear = nn.Linear(self.input_dim, self.input_dim)

#         # Define the LSTM layer
#         self.lstm = eval('nn.' + rnn_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

#         self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

#     def init_hidden(self):
#         return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
#                 torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

#     def forward(self, input):
#         linear_input = f.relu(self.init_linear(input))

#         # Forward pass through LSTM layer
#         # shape of lstm_out: [batch_size, input_size ,hidden_dim]
#         # shape of self.hidden: (a, b), where a and b both
#         # have shape (batch_size, num_layers, hidden_dim).
#         lstm_out, self.hidden = self.lstm(linear_input)
        

#         # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
#         y_pred = self.linear(lstm_out)
#         return y_pred

In [28]:
class BILSTMForNER(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=1, dropout=0.33):
        super(BILSTMForNER, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim * 2, 128)  # Linear output dim is 128
        self.classifier = nn.Linear(128, output_dim)  # Adjusted to match the number of unique NER tags
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))  # Apply dropout to embeddings
        lstm_output, _ = self.lstm(embedded)
        out = self.fc(lstm_output)
        out = F.elu(out)  
        out = self.classifier(out)
        return out


In [29]:
BILSTMForNER = BILSTMForNER(input_dim = 100, hidden_dim = 256, embedding_dim = 1024, output_dim = 128)

In [30]:
# class NERDataset(Dataset):
#     def __init__(self, file_path):
#         self.examples = []
#         self._load_data(file_path)

#     def _load_data(self, file_path):
#         with open(file_path, encoding='utf-8') as f:
#             current_sentence = []
#             for line in f:
#                 line = line.strip()
#                 if line: 
#                     _, word, tag = line.split()
#                     current_sentence.append((word, tag))
#                 else: 
#                     if current_sentence:  # Avoid adding empty sentences
#                         self.examples.append(current_sentence)
#                         current_sentence = []
#             # Add the last sentence if file doesn't end with a newline
#             if current_sentence:
#                 self.examples.append(current_sentence)

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, idx):
#         words, tags = zip(*self.examples[idx])
#         return {"tokens": words, "tags": tags}

# def collate_fn(batch):
#     """
#     A custom collate function to pad the sequences to the same length in each batch.
#     """
#     max_length = max(len(item["tokens"]) for item in batch)
#     padded_tokens, padded_tags = [], []
#     for item in batch:
#         padding_length = max_length - len(item["tokens"])
#         padded_tokens.append(list(item["tokens"]) + ['<PAD>']*padding_length)
#         padded_tags.append(list(item["tags"]) + ['O']*padding_length)  # Assuming 'O' is the padding tag
#     return {"tokens": torch.tensor(padded_tokens), "tags": torch.tensor(padded_tags)}


In [31]:
# def task1_collate(data: list[dict], use_targets: bool) -> dict[str, torch.Tensor]:
#     word_encodings = [torch.Tensor(x['word_encodings']) for x in data]
#     word_encodings = pad_sequence(word_encodings, batch_first=True)

#     lengths = torch.IntTensor([len(x['word_encodings']) for x in data])

#     # capital_mask = [torch.Tensor(x['capital_mask']) for x in data]
#     # capital_mask = pad_sequence(capital_mask, batch_first = True)

#     item: dict[str, torch.Tensor] = {
#         'word_encodings': word_encodings,
#         'lengths': lengths,
#     }

#     if use_targets: 
#         tag_encodings = [torch.Tensor(x['tag_encodings']) for x in data] # type: ignore
#         tag_encodings = pad_sequence(tag_encodings, batch_first=True)
#         item |= {'tag_encodings': tag_encodings}
    
#     return item
    

In [32]:
def train_model(model, dataloaders, optimizer, criterion, device, num_epochs=50, patience=10):
    """
    Trains and validates the model.
    
    Args:
    - model (torch.nn.Module): The PyTorch model to train.
    - dataloaders (dict): A dictionary containing 'train' and 'val' DataLoaders.
    - optimizer (torch.optim.Optimizer): The optimizer to use for training.
    - criterion (torch.nn.Module): The loss function.
    - num_epochs (int): The number of epochs to train for.
    - patience (int): The patience for early stopping.
    """
    best_val_f1 = -float('inf')  
    patience_counter = 0
    model.to(device)
    best_epoch = -1
    
    for epoch in range(num_epochs):
        
        train_preds, train_labels, val_preds, val_labels = [], [], [], []

        model.train()
        for batch in dataloaders['train']:

            inputs, labels, _ = batch['tokens'].to(device), batch['tags'].to(device).unsqueeze(-1)
            optimizer.zero_grad()
            outputs = model(inputs)
            t_preds = (outputs > pred_threshold).long() 

            # print('outputs: ', list(outputs.detach().cpu().numpy()),'labels: ', list(labels.detach().cpu().numpy()) )
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate training acc.
            train_preds.extend(t_preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        train_acc = np.mean(np.array(train_preds) == np.array(train_labels))

        model.eval()
        with torch.no_grad():
            for batch in dataloaders['val']:
                inputs, labels = batch['tokens'].to(device), batch['tags'].to(device)

                outputs = model(inputs)
                preds = (outputs > pred_threshold).long() 
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())


        val_accuracy = np.mean(np.array(val_preds) == np.array(val_labels))
        val_f1 = f1_score(val_labels, val_preds)
        
        if val_f1 > best_val_f1:
            best_val_f1, best_epoch = val_f1, epoch 
            patience_counter = 0
            # print(f"Validation F1 improved. Saving model...")
            torch.save(model.state_dict(), f'saved_models/epoch:{epoch}_valacc:{val_accuracy}_checkpoint.pth')
        else:
            patience_counter += 1 
            # print(f'Validation F1 did not improve. Patience: {patience_counter}/{patience}')
        
        # Early stopping check
        if patience_counter >= patience:
            print("Early stopping triggered")
            break  


    # print(f'Validation Accuracy: {val_accuracy:.4f}. Training Accuracy: {train_acc:.4f}')
    return val_accuracy,train_acc, best_val_f1, np.array(train_preds), np.array(train_labels), np.array(val_preds), np.array(val_labels)


In [33]:
class IndexedNERDataset(Dataset):
    def __init__(self, file_path):
        super().__init__()
        self.sentences = []
        self.tags = []
        self._load_data(file_path)
        
    def _load_data(self, file_path):
        with open(file_path, encoding='utf-8') as f:
            sentence, tag_sequence = [], []
            for line in f:
                line = line.strip()
                if line:
                    idx, word_idx, tag_idx = line.split()
                    sentence.append(word_idx)
                    tag_sequence.append(tag_idx)
                else:  # End of sentence
                    self.sentences.append(sentence)
                    self.tags.append(tag_sequence)
                    sentence, tag_sequence = [], []
            # Adding last sentence if file doesn't end with a newline
            if sentence and tag_sequence:
                self.sentences.append(sentence)
                self.tags.append(tag_sequence)
                
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        # For dynamic padding, return sentences and tags as they are
        print(self.sentences[idx])
        sentence = torch.tensor(self.sentences[idx], dtype=torch.long)
        tag_sequence = torch.tensor(self.tags[idx], dtype=torch.long)
        return sentence, tag_sequence

def collate_fn(batch):
    (xx, yy) = zip(*batch)
    
    # Pad sequences to the longest sequence in the batch
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)  # Assuming 0 is the padding index for tokens
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=-1) # Assuming -1 is the padding index for tags

    # You can also compute a mask here, indicating padded areas
    mask = torch.tensor([[float(i != -1) for i in seq] for seq in yy_pad], dtype=torch.float)

    return xx_pad, yy_pad, mask



Setup hyperparams for training loop 

In [34]:
train_dataset = IndexedNERDataset('../../data/lstm-data/train')
dev_dataset = IndexedNERDataset('../../data/lstm-data/dev')

dataloaders = {}

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, collate_fn = collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size = 32, shuffle=False, collate_fn = collate_fn)
dataloaders['train'], dataloaders['val'] = train_loader, dev_loader

optimizer = optim.Adam(BILSTMForNER.parameters(),lr = 0.001 )
criterion = nn.CrossEntropyLoss()

In [35]:
val_accuracy, train_acc, best_val_f1, train_preds, train_labels, val_preds, val_labels = train_model(BILSTMForNER, dataloaders, optimizer, criterion, device,  num_epochs=50, patience=10)

['3', '-', 'Jana', 'Novotna', '(', 'Czech', 'Republic', ')', 'beat', 'Sandrine', 'Testud', '(', 'France', ')']


ValueError: too many dimensions 'str'

Task 2: Using GloVe word embeddings
Task is to use the GloVe word embeddings to improve the BLSTM
in Task 1. The way we use the GloVe word embeddings is straight forward:
we initialize the embeddings in our neural network with the corresponding
vectors in GloVe. Note that GloVe is case-insensitive, but our NER model
should be case-sensitive because capitalization is an important information
for NER. You are asked to find a way to deal with this conflict. What are
the precision, recall and F1 score on the dev data? (hint: the reasonable F1
score on dev is 88%.

Bonus: LSTM-CNN model 
The bonus task is to equip the BLSTM model in Task 2 with a CNN module
to capture character-level information (see slides page 45 in lecture 12 for the
network architecture). The character embedding dimension is set to 30. You
need to tune other hyper-parameters of CNN module, such as the number of
CNN layers, the kernel size and output dimension of each CNN layer. What
are the precision, recall and F1 score on the dev data? Predicting the NER
tags of the sentences in the test data and output the predictions in a file
named pred, in the same format of training data. (hint: the bonus points are
assigned based on the ranking of your model F1 score on the test data).